In [ ]:
import pandas as pd
import numpy as np

def wiperonly(file):
    df1= file[file.productName.str.contains(r'wiper')]
    df2= file[file.productName.str.contains(r'Wiper')]
    df3= file[file.productName.str.contains(r'WIPER')]
    data= df1.append([df2, df3],ignore_index = True).drop_duplicates()
    #data= df1.merge(df2, how='outer').drop_duplicates()
    #data= data.merge(df3, how='outer').drop_duplicates()
    return data

def wiperdata(country, year, ccode):
  if (year== "2019"):
    file= pd.read_csv("/content/drive/MyDrive/Bosch/2019/08_December 2019/Lazada/02_QnA/2020-01-01_Lazada_" +country+"_questions_items.csv")
    data= wiperonly(file)
    data.to_csv("/content/drive/MyDrive/Bosch/2019/08_December 2019/Lazada/02_QnA/Analysis/"+ccode+year+"_wiperqna.csv" , index=False)
    return data
  elif (year=="2020"):
    file= pd.read_csv("/content/drive/MyDrive/Bosch/2020/16_Oct_2020/Lazada/03_Q&A/2020-11-01_Lazada_" + country + "_questions_items.csv")
    data= wiperonly(file)
    data.to_csv("/content/drive/MyDrive/Bosch/2020/16_Oct_2020/Lazada/03_Q&A/Analysis/" +ccode+year+"_wiperqna.csv", index=False)
    return data

#countryCodes: ['INDO', 'MSIA', 'PHI', 'SG', 'THAI', 'VIET']
#parameters (platform, country, year, ccode)
#2020 Lazada Datasets
indo2019 = wiperdata("Indonesia", "2019", "INDO" )
msia2019 = wiperdata("Malaysia", "2019", "MSIA")
phi2019 = wiperdata("Philippines", "2019", "PHI")
sin2019 = wiperdata("Singapore", "2019", "SG")
thai2019 = wiperdata("Thailand", "2019", "THAI")
viet2019 = wiperdata("Vietnam", "2019", "VIET")

#2020 Lazada Datasets
indo2020 = wiperdata("Indonesia", "2020", "INDO" )
msia2020 = wiperdata("Malaysia", "2020", "MSIA")
phi2020 = wiperdata("Philippines", "2020", "PHI")
sin2020 = wiperdata("Singapore", "2020", "SG")
thai2020 = wiperdata("Thailand", "2020", "THAI")
viet2020 = wiperdata("Vietnam", "2020", "VIET")

In [ ]:
boschWipers = pd.read_excel("/content/drive/MyDrive/Bosch/Wipers__Asean.xlsx")
boschWipers['After sales short term'] = boschWipers['After sales short term'].str.replace(' ', '')

In [ ]:
#to identify which are bosch, we will use the following:

import re

#relevantNrs are just long numbers, but asst may have variations that we must also check for

#most product codes follow the format ABC 123 D / ABC 123, with some only being numerals
#need to account for all possibilities: ABC123D, ABC 123D, ABC123 D, ABC 123 D, ABC-123D, ABC123-D, ABC-123-D

#standardise the original bosch codes by removing all spaces
def strip_all_except_alphanumeric(product_name):
  # strip all characters except alphanumeric characters
  # reference: 
  # https://stackoverflow.com/questions/1276764/stripping-everything-but-alphanumeric-chars-from-a-string-in-python
  pattern = re.compile('[\W_]+', re.UNICODE)
  return pattern.sub('', str(product_name))

#the following function will be used to add spaces between numbers and alphabets in the product code:
#https://stackoverflow.com/questions/48963908/how-to-insert-space-between-alphabet-characters-and-numeric-character-using-rege
def add_spaces(product_code):
  resultList = []
  regexAll = r"(?i)(?<=\d)(?=[a-z])|(?<=[a-z])(?=\d)" #adds all spaces (ABC123D → ABC 123 D)
  regexCharNum = r"(?i)(?<=\d)(?=[a-z])" #ABC123D → ABC123 D
  regexNumChar = r"(?i)(?<=[a-z])(?=\d)" #ABC123D → ABC 123D
  insertSpace = " "
  insertDash = "-"

  result = re.sub(regexAll, insertSpace, product_code, 0)
  if result != product_code:
      resultList.append(result)
  result = re.sub(regexAll, insertDash, product_code, 0)
  if result != product_code:
      resultList.append(result)

  result = re.sub(regexCharNum, insertSpace, product_code, 0)
  if result != product_code:
      resultList.append(result)
  result = re.sub(regexCharNum, insertDash, product_code, 0)
  if result != product_code:
      resultList.append(result)

  result = re.sub(regexNumChar, insertSpace, product_code, 0)
  if result != product_code:
      resultList.append(result)

  result = re.sub(regexNumChar, insertDash, product_code, 0)
  if result != product_code:
      resultList.append(result)

  if resultList != []: 
    return resultList
  else:
    return None

def getAllASST(uniqueASST):  
  allPossibleASST = np.copy(uniqueASST) #not sure if need this
  index = 0
  for asst in allPossibleASST:
    allPossibleASST[index] = strip_all_except_alphanumeric(asst)
    index += 1

  #keep a count of the original codes. we will loop through only these
  numOriginalCodes = len(uniqueASST)
  for index in range(numOriginalCodes):
    newCode = add_spaces(allPossibleASST[index])
    if newCode is not None:
      allPossibleASST = np.append(allPossibleASST, newCode)

  return allPossibleASST

In [ ]:
#https://stackoverflow.com/questions/5319922/python-check-if-word-is-in-a-string
def findWholeWord(w):
    return re.compile(r'\b({0})\b'.format(w), flags=re.IGNORECASE).search

def is_bosch(productName, allPossibleASST, uniqueRelevantNrs):
  # if we are able to find the Relevant NR or ASST code in the product name,
  # we consider it a Bosch item and set isBosch=True
  print("checking: ", productName)
  productName = str(productName)
  for asst in allPossibleASST:
    # print("asst: ", productName, asst, findWholeWord(asst)(productName))
    if findWholeWord(asst)(productName): 
        if asst in ['500', '600', '650']: #prevent issues like productcode 500 being recognised in listings with 500 mm
          if (findWholeWord(asst + ' mm')(productName) or findWholeWord(asst + ' cm')(productName)):
            continue
        print(productName, 'contains', asst)
        return True
  for num in uniqueRelevantNrs:
    # print("relevant num: ", productName, num, findWholeWord(num)(productName))
    if (findWholeWord(num)(productName)):
      print(productName, 'contains', num)
      return True

  return False


  #just append the relevant nr prod text

In [ ]:
#append prod key text for bosch products

def get_lookup_category(boschCSV):
  lookupCategory = boschCSV[['Relevant Nr', 'After sales short term', 'Relevant Nr prod key text']]
  #standardise bosch codes
  lookupCategory['After sales short term'] = lookupCategory['After sales short term'].apply(strip_all_except_alphanumeric)
  #drop any duplicates
  lookupCategory = lookupCategory.drop_duplicates()

  return lookupCategory

#only run this on rows where isBosch == True
def append_prod_text(productName, allPossibleASST, uniqueRelevantNrs, lookupCategory):
  result = []
  productName = str(productName)
  for asst in allPossibleASST:
    # print("asst: ", productName, asst, findWholeWord(asst)(productName))
    if findWholeWord(asst)(productName): 
        if asst in ['500', '600', '650']: #prevent issues like productcode 500 being recognised in listings with 500 mm
          if (findWholeWord(asst + ' mm')(productName) or findWholeWord(asst + ' cm')(productName)):
            continue
        print(productName, 'contains', asst)
        standardisedASST = strip_all_except_alphanumeric(asst)
        #print(lookupCategory[lookupCategory['After sales short term'] == standardisedASST].squeeze()) #['Relevant Nr prod key text'].item()
        result.append(lookupCategory[lookupCategory['After sales short term'] == standardisedASST]['Relevant Nr prod key text'].item())
        result.append(lookupCategory[lookupCategory['After sales short term'] == standardisedASST]['Relevant Nr'].item())
        result.append(lookupCategory[lookupCategory['After sales short term'] == standardisedASST]['After sales short term'].item())
        return result
        # return lookupCategory[lookupCategory['After sales short term'] == standardisedASST]
  for num in uniqueRelevantNrs:
     #print("relevant num: ", productName, num, findWholeWord(num)(productName))
     if (findWholeWord(num)(productName)):
      print(productName, 'contains', num)
      # print("CHECKING: " , lookupCategory[lookupCategory['Relevant Nr'] == num]['Relevant Nr prod key text'])
      # print(lookupCategory[lookupCategory['Relevant Nr'] == num].squeeze()) #['Relevant Nr prod key text'].item()
      # return lookupCategory[lookupCategory['Relevant Nr'] == num]
      result.append(lookupCategory[lookupCategory['Relevant Nr'] == num]['Relevant Nr prod key text'].item())
      result.append(lookupCategory[lookupCategory['Relevant Nr'] == num]['Relevant Nr'].item())
      result.append(lookupCategory[lookupCategory['Relevant Nr'] == num]['After sales short term'].item())
      print(result)
      # return (lookupCategory[lookupCategory['Relevant Nr'] == num]['Relevant Nr prod key text'].item(), lookupCategory[lookupCategory['Relevant Nr'] == num]['Relevant Nr'].item(),)
      return result

  return [np.nan, np.nan, np.nan] #false?

In [ ]:
def extract_data(data): 
  data['Relevant Nr prod key text'] = data['Relevant Nr prod key text'].apply(lambda x: x[0] if isinstance(x, list) else x)
  data['Relevant Nr'] = data['Relevant Nr'].apply(lambda x: x[1] if isinstance(x, list) else x)
  data['ASST'] = data['ASST'].apply(lambda x: x[2] if isinstance(x, list) else x)
  
def identify_subcategory(data, allPossibleASST, uniqueRelevantNrs, lookupCategory):
  data[['Relevant Nr prod key text', 'Relevant Nr', 'ASST']] = [np.nan, np.nan, np.nan]
  data.loc[data['isBosch'], ['Relevant Nr prod key text', 'Relevant Nr', 'ASST']] = data[data['isBosch']]['productName'].apply(lambda text: append_prod_text(text, allPossibleASST, uniqueRelevantNrs, lookupCategory))
  extract_data(data)

In [ ]:
def shortcodesearch(file):
  temp_df= pd.DataFrame(columns= ['productName', 'brand', 'question', 'questionTimestamp', 'answer',
       'responseTime', 'soldBy', 'sku', 'productUrl', 'categories', 'isBosch',
       'Relevant Nr prod key text_x', 'Relevant Nr_x', 'ASST', 'RG', 'Brand',
       'Segment', 'MV Nr', 'Vehicle typ', 'Maintenance mod range',
       'Internal mod range', 'ESI mod range', 'Product', 'Relevant Nr_y',
       'Relevant Nr prod key text_y', 'Unnamed: 11', 'After sales short term'])
  for i in range(len(file)):
    data= file.loc[i]
    carmodel= file.Brand[i]
    question= file.question[i]
    upper= question.find(carmodel)
    lowercarmodel= carmodel.lower()
    lower= question.find(lowercarmodel)
    oneupper= carmodel[0]+lowercarmodel[1:]
    oneupper_find= question.find(oneupper)
    if (upper!= -1 or lower!=-1 or oneupper_find!=-1):
      temp_df= temp_df.append(data)
  return (temp_df)

In [ ]:
lookcat= get_lookup_category(boschWipers)
incorrect_data = [38327, 13890, 1665, 184, 52, 842, 36613, 14565, 36644, 29841, 33588, 7526, 10686, 8164, 29034, 6676, 7502, 7535, 7574, 7518, 7575, 834, 2444, 185, 922, 783, 1108, 13633, 795, 324, 117, 313, 7949, 881, 29144, 3402, 20673, 7708, 38202, 20351, 3410, 2957, 539, 3520, 293, 4273, 15269, 247, 1869, 949, 2924, 3779, 1743, 4622, 1228, 238, 1083, 1187, 3521, 1229, 33955, 926, 294, 3406, 4334, 3352, 351, 295, 8930, 17745, 3975, 2409, 315, 14040, 9, 9880, 32, 658, 827, 1, 22606, 892, 34, 3094, 916, 11597, 396, 1668, 7267, 1670, 326, 57, 1084, 629, 304, 924, 7269, 4094, 935, 3750, 1673, 1679, 317, 2, 815, 3066, 308, 820, 22608, 18, 3095, 920, 1674, 2873, 7165, 320, 4, 321, 638, 13, 22609, 14, 1599, 20, 1645, 13779, 13780, 17726, 13773, 8799, 8798, 14133, 8759]
lookcat = lookcat.drop(index=incorrect_data)
lookcat= lookcat.reset_index(drop=True)
allPossibleASST= lookcat['After sales short term']
uniqueRelevantNrs= lookcat['Relevant Nr']
lookupCategory= lookcat['Relevant Nr prod key text']
lookupCategory= lookupCategory.drop(index=incorrect_data).reset_index(drop=True)

# Indonesia 

Here's a sample of how to run the code

In [ ]:
#indo2020
indo2020['isBosch']= indo2020['productName'].apply(lambda productName : is_bosch(productName, allPossibleASST, uniqueRelevantNrs))
identify_subcategory(indo2020,allPossibleASST, uniqueRelevantNrs, lookcat)
indo2020= indo2020.dropna(subset= ['ASST']).reset_index(drop=True)
indo2020merge = pd.merge(indo2020, boschWipers, left_on='ASST', right_on='After sales short term')
#for indo2020
indo2020merge_car=shortcodesearch(indo2020merge)
indo2020merge_car= indo2020merge_car.reset_index(drop=True)
new = indo2020merge_car["Relevant Nr prod key text_x"].str.split(",", n = 1, expand = True) 
indo2020merge_car["Relevant Nr prod key text_x"]= new[0]
indo2020merge_car["Relevant Nr prod key text_y"]= new[1]
indo2020merge_car=indo2020merge_car.drop_duplicates(subset=['productName', 'question', 'After sales short term', 'Brand'], keep='first').reset_index(drop=True)

In [ ]:
g= indo2020merge_car.groupby('Relevant Nr prod key text_x')
g.ngroups
g.apply(lambda x: x.to_csv(r'/content/drive/MyDrive/Bosch/QnA stuff/Data/Indonesia/2020/indo2020_{}.csv'.format(x.name), index=False))

In [ ]:
import os

##########
directory = '/content/drive/MyDrive/Bosch/QnA stuff/Data/Indonesia/2020'
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
      print(filename)
      file= pd.read_csv("/content/drive/MyDrive/Bosch/QnA stuff/Data/Indonesia/2020/"+filename)
      valcount= file.value_counts(subset=['Brand'])
      print(valcount)
      carmodels= valcount.index
      listofsubcat=[]
      for i in range(len(carmodels)):
        car= carmodels[i][0]
        listofsubcat.append(car)
      sub= file['Relevant Nr prod key text_x'][0]
      df= pd.DataFrame(columns= ['Brand','subcat', 'frequency'])
      for x in range(len(listofsubcat)):
        brand= listofsubcat[x]
        freq= valcount[x]
        df = df.append(
            {'Brand': brand,
            'subcat': sub,
              'frequency': freq
              }, ignore_index=True)
      if (df is not None):
        #print(df)
        name= "/content/drive/MyDrive/Bosch/QnA stuff/Indonesia/"+filename
        name= str(name)
        #print(name)
        df.to_csv(name, index=False)

print('done')

# Philippines

In [ ]:
#phi2020
phi2020['isBosch']= phi2020['productName'].apply(lambda productName : is_bosch(productName, allPossibleASST, uniqueRelevantNrs))
identify_subcategory(phi2020,allPossibleASST, uniqueRelevantNrs, lookcat)
phi2020_merged= phi2020.dropna(subset= ['ASST']).reset_index(drop=True)
phi2020merge = pd.merge(phi2020, boschWipers, left_on='ASST', right_on='After sales short term')
phi2020merge_car=shortcodesearch(phi2020merge)
phi2020merge_car= phi2020merge_car.reset_index(drop=True)
new = phi2020merge_car["Relevant Nr prod key text_x"].str.split(",", n = 1, expand = True) 
phi2020merge_car["Relevant Nr prod key text_x"]= new[0]
phi2020merge_car["Relevant Nr prod key text_y"]= new[1]
phi2020merge_car=phi2020merge_car.drop_duplicates(subset=['productName', 'question', 'After sales short term', 'Brand'], keep='first').reset_index(drop=True)

In [ ]:
g= phi2020merge_car.groupby('Relevant Nr prod key text_x')
g.ngroups
g.apply(lambda x: x.to_csv(r'/content/drive/MyDrive/Bosch/QnA stuff/Data/Philippines/2020/phi2020_{}.csv'.format(x.name), index=False))

In [ ]:
import os

##########
directory = '/content/drive/MyDrive/Bosch/QnA stuff/Data/Philippines/2020'
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
      print(filename)
      file= pd.read_csv("/content/drive/MyDrive/Bosch/QnA stuff/Data/Philippines/2020/"+filename)
      valcount= file.value_counts(subset=['Brand'])
      print(valcount)
      carmodels= valcount.index
      listofsubcat=[]
      for i in range(len(carmodels)):
        car= carmodels[i][0]
        listofsubcat.append(car)
      sub= file['Relevant Nr prod key text_x'][0]
      df= pd.DataFrame(columns= ['Brand','subcat', 'frequency'])
      for x in range(len(listofsubcat)):
        brand= listofsubcat[x]
        freq= valcount[x]
        df = df.append(
            {'Brand': brand,
            'subcat': sub,
              'frequency': freq
              }, ignore_index=True)
      if (df is not None):
        #print(df)
        name= "/content/drive/MyDrive/Bosch/QnA stuff/Philippines/"+filename
        name= str(name)
        #print(name)
        df.to_csv(name, index=False)

print('done')